In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.naive_bayes import GaussianNB

In [4]:
import warnings

In [5]:
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('/content/sample_data/mnist_train_small.csv', header = None)

In [7]:
df.shape

(20000, 785)

In [8]:
df.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
18613,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9318,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8071,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6581,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2480,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9131,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4979,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10176,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15635,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7879,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 785 entries, 0 to 784
dtypes: int64(785)
memory usage: 119.8 MB


In [10]:
features = df.iloc[:,1:785]

In [11]:
features.head()

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
labels = df.iloc[:,0]

In [13]:
labels.head()

0    6
1    5
2    7
3    9
4    5
Name: 0, dtype: int64

In [ ]:
cross_val_score(KNeighborsClassifier(), features, labels, cv = 10, scoring = 'accuracy').mean()

0.95935

In [ ]:
cross_val_score(DecisionTreeClassifier(), features, labels, cv = 10, scoring = 'accuracy').mean()

0.8368500000000001

In [ ]:
cross_val_score(DecisionTreeClassifier(max_depth = 5), features, labels, cv = 10, scoring = 'accuracy').mean()

0.6909500000000001

In [ ]:
cross_val_score(DecisionTreeClassifier(max_depth = 10), features, labels, cv = 10, scoring = 'accuracy').mean()

0.83675

In [ ]:
cross_val_score(SVC(), features, labels, cv = 10, scoring = 'accuracy').mean()

0.96905

In [ ]:
cross_val_score(GaussianNB(), features, labels, cv = 10, scoring = 'accuracy').mean()

0.56735

## I'm choosing ***Gaussian Naive Bayes*** as a **fitness function** as it give lower accuracy than any other models and this gives same importance to all the features. So we've to choose a feature set which gives better accuracy.

In [ ]:
class simple_genetic_algorithm:
    
    def __init__(self, features, labels, population_size: int, chromosome_size: int, crossover_prob: float = 0.6, mutation_prob: float = 0.01, generation: int = 10):
        
        self.features = features
        self.labels = labels
        self.popsize = population_size
        self.chromsize = chromosome_size
        self.crossprob = crossover_prob
        self.mutation_prob = mutation_prob
        self.generation = generation
        self.population = []
        self.fit_score = {}
        self.chfit_score = {}
        self.mating_pool = []
        self.initial_pop()
        self.solution = []
        
        for i in range(self.generation):
            self.fitness_gen()
            print(f"best fitness value of generation {i} = {max(self.chfit_score.keys())}")
            self.binary_tournament()
            self.crossing_over()
        self.bestsolution = self.population[self.chfit_score[max(self.chfit_score.keys())]]
        
        for gene in range(self.chromsize):
                if self.bestsolution[gene] == 0:
                    continue
                else:
                    self.solution.append(gene)

    def initial_pop(self):
        
        for i in range(self.popsize):
            chrom = np.random.randint(2 ,size=self.chromsize)
            self.population.append(chrom)

    def fitness_gen(self):
        
        self.fit_score = {}
        self.chfit_score = {}
        
        for chromosome in range(self.popsize):
            feature_pos = self.population[chromosome]
            feature_set = []
            
            for gene in range(self.chromsize):
               
                if feature_pos[gene] == 0:
                    continue
                else:
                    feature_set.append(gene)


            chrom_fit_score = cross_val_score(GaussianNB(), self.features.iloc[:, feature_set], self.labels, cv = 10, scoring = 'accuracy').mean()
            self.fit_score[chromosome] = chrom_fit_score
            self.chfit_score[chrom_fit_score] = chromosome
    
    def elitism(self, gen):
        
        fit = {}
        
        for chromosome in range(self.popsize):
            feature_pos = self.population[chromosome]
            feature_set = []
            
            for gene in range(self.chromsize):
                
                if feature_pos[gene] == 0:
                    continue
                else:
                    feature_set.append(gene)


            chrom_fit_score = cross_val_score(GaussianNB(), self.features.iloc[:, feature_set], self.labels, cv = 10, scoring = 'accuracy').mean()
            fit[chrom_fit_score] = chromosome
        
        if max(self.chfit_score.keys()) > min(fit.keys()):
            weakest = fit[min(fit.keys())]
            gen[weakest] = self.population[self.chfit_score[max(self.chfit_score.keys())]]
        
        return gen


    def binary_tournament(self):
        
        self.mating_pool = []
        sample1 = np.random.randint(self.popsize)
        sample2 = np.random.randint(self.popsize)
        
        while len(self.mating_pool) != self.popsize :
            
            if self.fit_score[sample1] >= self.fit_score[sample2]:
                self.mating_pool.append(self.population[sample1])
            else:
                self.mating_pool.append(self.population[sample2])
    
    def mutation(self, chromosome: list)->list:
        
        for gene_position in range(self.chromsize):
            
            random_number = np.random.random()
            
            if random_number <= self.mutation_prob:
                
                if chromosome[gene_position] == 1:
                    chromosome[gene_position] = 0
                else:
                    chromosome[gene_position] = 1
        
        return chromosome
    
    def crossing_over(self):
        
        new_gen = []
        
        while len(new_gen) != self.popsize:
            
            cross_prob = np.random.random()
            parent1 = self.mating_pool[np.random.randint(self.popsize)]
            parent2 = self.mating_pool[np.random.randint(self.popsize)]

            if cross_prob <= self.crossprob:
                
                child1 = np.concatenate((parent1[:self.chromsize//2], parent2[self.chromsize//2:]))
                child2 = np.concatenate((parent2[:self.chromsize//2], parent1[self.chromsize//2:]))
                mut_child1 = self.mutation(child1)
                mut_child2 = self.mutation(child2)
                new_gen.append(child1)
                new_gen.append(child2)
            else:
                new_gen.append(parent1)
                new_gen.append(parent2)
        
        new_gen = self.elitism(new_gen)
        self.population = new_gen
    

In [ ]:
ga = simple_genetic_algorithm(features, labels, population_size= 20, chromosome_size= 784)

best fitness value of generation 0 = 0.5899999999999999
best fitness value of generation 1 = 0.5990499999999999
best fitness value of generation 2 = 0.6002
best fitness value of generation 3 = 0.60575
best fitness value of generation 4 = 0.6067500000000001
best fitness value of generation 5 = 0.6096
best fitness value of generation 6 = 0.6183
best fitness value of generation 7 = 0.6183
best fitness value of generation 8 = 0.6183
best fitness value of generation 9 = 0.6183


In [ ]:
len(ga.solution)

407

In [ ]:
ga.solution

[0,
 1,
 2,
 5,
 6,
 12,
 13,
 15,
 16,
 17,
 19,
 21,
 23,
 29,
 30,
 31,
 35,
 38,
 39,
 41,
 44,
 46,
 47,
 49,
 50,
 51,
 54,
 55,
 57,
 58,
 59,
 61,
 63,
 64,
 65,
 68,
 69,
 71,
 77,
 79,
 82,
 86,
 87,
 90,
 91,
 92,
 96,
 97,
 98,
 100,
 103,
 107,
 108,
 112,
 116,
 122,
 124,
 125,
 126,
 128,
 129,
 131,
 132,
 135,
 136,
 138,
 139,
 142,
 145,
 146,
 150,
 153,
 154,
 156,
 159,
 161,
 163,
 164,
 165,
 167,
 168,
 169,
 170,
 173,
 175,
 176,
 177,
 178,
 179,
 180,
 182,
 183,
 185,
 186,
 188,
 189,
 190,
 191,
 194,
 196,
 203,
 204,
 206,
 208,
 209,
 212,
 215,
 216,
 217,
 218,
 219,
 220,
 222,
 224,
 227,
 228,
 229,
 230,
 234,
 235,
 236,
 237,
 238,
 240,
 242,
 243,
 246,
 247,
 248,
 249,
 250,
 257,
 258,
 260,
 261,
 262,
 264,
 266,
 268,
 269,
 270,
 271,
 273,
 274,
 277,
 278,
 279,
 282,
 285,
 287,
 288,
 291,
 293,
 294,
 297,
 298,
 300,
 302,
 303,
 305,
 308,
 309,
 311,
 312,
 313,
 315,
 320,
 322,
 324,
 325,
 327,
 329,
 330,
 331,
 332,
 333

In [20]:
class compact_genetic_algorith:
    def __init__(self, features, labels, epochs, chromosome_size):
        
        self.features = features
        self.labels = labels
        self.epochs = epochs
        self.chromsize = chromosome_size
        self.prob = self.initializing_probabilty()
        self.update_prob()
        

    def initializing_probabilty(self):
        return np.full(self.chromsize, 0.5)

    def generating_chromosome(self):
        return np.random.randint(2, size = self.chromsize)
    
    def fitness_gen(self, chrom):
        feature_set = []
        for gene in range(self.chromsize):
            if chrom[gene] == 0:
                continue
            else:
                feature_set.append(gene)

        chrom_fit_score = cross_val_score(GaussianNB(), self.features.iloc[:, feature_set], self.labels, cv = 10, scoring = 'accuracy').mean()
        return chrom_fit_score

    def update_prob(self):

        for i in range(self.epochs):
        
            chrom1 = self.generating_chromosome()
            chrom2 = self.generating_chromosome()
            fit1 = self.fitness_gen(chrom1)
            fit2 = self.fitness_gen(chrom2)
            
            if fit1 > fit2:
                winner = chrom1
                loser = chrom2
            else:
                winner = chrom2
                loser = chrom1

            for chrom in range(self.chromsize):

                if winner[chrom] != loser[chrom]:
                    if winner[chrom] == 1:
                        self.prob[chrom] = self.prob[chrom] + (1/self.chromsize)
                    else:
                        self.prob[chrom] = self.prob[chrom] - (1/self.chromsize)


In [26]:
p = compact_genetic_algorith(features, labels, 100, 784)

In [29]:
ppp = p.prob

In [30]:
feature_set = []
for i in range(784):
    if ppp[i] < 0.5:
        continue
    else:
        feature_set.append(i)

In [31]:
cross_val_score(GaussianNB(), features.iloc[:, feature_set], labels, cv = 10, scoring = 'accuracy').mean()

0.6609499999999999

#Compact Genetic Algorithm is better than Simple Genetic Algorithm.
<br> It gives better accuracy in less amount of time.

<br> [1] In this problem ***Simple Genetic Algorithm*** took 15 minutes to execute(only 10 generations) and gives an accuracy of **61.83%** which initially **56.74%** without any feature selection.

<br> [2] On the other hand ***Compact Genetic Algorithm*** took only 7 minutes to execute(100 epochs) and gives an accuracy of **66.09%** which initially **56.74%** without any feature selection.